In [ ]:
import os
gpu_device = 1    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd
import random
import math
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict
import re
import numpy as np

In [ ]:
model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

In [ ]:
#Mejor con 4 epochs i 5e-5

training_args= TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=4,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=5e-5, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

In [ ]:
%run ../utils/data.ipynb
%run ../utils/BERT_retrieval.ipynb

In [ ]:
accuracies,output_dic,output_dic_t=cross_validation(dataset_final,targets)

In [ ]:
model_name="bioMedBERT"+"5e55"+"16batch"
for tipo in questions:
    print(tipo)
    with open(f"../Generativos/results_dic_{tipo}/{model_name.split("/")[-1]}.pkl", "wb") as file:
        pickle.dump(output_dic[tipo], file)
    with open(f"../Generativos/results_dic_{tipo}/{model_name.split("/")[-1]}truth.pkl", "wb") as file:
        pickle.dump(output_dic_t[tipo], file)
   

In [ ]:
def calculate_acc(pred,truth,point=False):
    acc=0
    total=0
    for key,real in truth.items():
        if key in pred:
            total+=1
            predicted=pred[key]    
            if point==True:

                predicted=predicted.split(".")[0]+"."

            elif point=="mm":
                predicted=predicted.split("mm")[0]+"mm."
            if predicted.lower()==str(real).lower():
                acc+=1
            # else:
            #     print(key,real,predicted)
        # else:
        #     print(key)
    return acc/total

In [ ]:
np.save("predicted_biomed_8epoch_truncate.npy", predicted)
np.save("tested_biomed_8epoch_truncate.npy", tested)